
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [ ]:
# File location and type
file_location = "/FileStore/tables/tips-1.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.csv(file_location, header=True, inferSchema=True)
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [ ]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [ ]:
df.columns

Out[3]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [ ]:
idx = ['sex', 'smoker', 'day', 'time']
O_idx = ["{}_idx".format(var) for var in idx]
O_idx

Out[13]: ['sex_idx', 'smoker_idx', 'day_idx', 'time_idx']

In [ ]:
### Handling Categorical Feature
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCols=idx, outputCols=O_idx)

In [ ]:
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-------+----------+-------+--------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_idx|smoker_idx|day_idx|time_idx|
+----------+----+------+------+---+------+----+-------+----------+-------+--------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|    1.0|       0.0|    1.0|     0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|    0.0|       0.0|    1.0|     0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|    0.0|       0.0|    1.0|     0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|    0.0|       0.0|    1.0|     0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|    1.0|       0.0|    1.0|     0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|    0.0|       0.0|    1.0|     0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|    0.0|       0.0|    1.0|     0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|    0.0|       0.0|    1.0|     0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|    0.0|       0.0|    1.0|  

In [ ]:
df_r.columns

Out[16]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_idx',
 'smoker_idx',
 'day_idx',
 'time_idx']

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['tip','size', 'sex_idx', 'smoker_idx', 'day_idx', 'time_idx'],
                            outputCol="Independent Feature")
output = assembler.transform(df_r)

In [ ]:
output.show()

+----------+----+------+------+---+------+----+-------+----------+-------+--------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_idx|smoker_idx|day_idx|time_idx| Independent Feature|
+----------+----+------+------+---+------+----+-------+----------+-------+--------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|    1.0|       0.0|    1.0|     0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|    0.0|       0.0|    1.0|     0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|    0.0|       0.0|    1.0|     0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|    0.0|       0.0|    1.0|     0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|    1.0|       0.0|    1.0|     0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|    0.0|       0.0|    1.0|     0.0|[4.71,4.0,0.0,0.0...|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|    0.0|

In [ ]:
output.select('Independent Feature').show()

+--------------------+
| Independent Feature|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [ ]:
final_data = output.select("Independent Feature", "total_bill")

final_data.show()

+--------------------+----------+
| Independent Feature|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [ ]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = final_data.randomSplit([0.75,0.25])

regressor = LinearRegression(featuresCol='Independent Feature', labelCol='total_bill')
regressor = regressor.fit(train_data)

In [ ]:
regressor.coefficients

Out[25]: DenseVector([3.194, 3.4898, -2.3432, 1.805, -0.2296, 0.2149])

In [ ]:
regressor.intercept

Out[26]: 1.5638811513506938

In [ ]:
### predictions
pred = regressor.evaluate(test_data)

In [ ]:
pred.predictions.show()

+--------------------+----------+------------------+
| Independent Feature|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.07| 12.53586386132539|
|(6,[0,1],[1.75,2.0])|     17.82|14.132844999807247|
| (6,[0,1],[2.0,2.0])|     12.69|14.931335569048176|
| (6,[0,1],[2.0,3.0])|     16.31|  18.4211005009332|
|(6,[0,1],[2.34,4.0])|     17.81|22.996812606985888|
|(6,[0,1],[2.64,3.0])|     17.59| 20.46523635818998|
|[1.0,1.0,1.0,0.0,...|      7.25| 5.904380748315061|
|[1.0,2.0,0.0,1.0,...|      12.6|13.542345022197347|
|[1.44,2.0,0.0,0.0...|      7.56| 12.89835123471325|
|[1.48,2.0,0.0,0.0...|      8.52|13.026109725791798|
|[1.5,2.0,1.0,0.0,...|     26.41|10.991126818681945|
|[1.5,2.0,1.0,0.0,...|     10.65|  10.7467613594467|
|[1.64,2.0,0.0,1.0...|     15.36|15.586480879454124|
|[1.96,2.0,0.0,0.0...|     15.04|14.573944303728894|
|[2.0,2.0,0.0,0.0,...|      8.77|14.701702794807442|
|[2.0,2.0,0.0,0.0,...|     13.81|14.7017027948

In [ ]:
# performance metrics
pred.meanAbsoluteError, pred.meanSquaredError, pred.r2

Out[29]: (4.0814773582795, 33.6621334382898, 0.4352018565916205)

In [ ]:
regressor.save("reagressorModel.pkl")